## Test Read SKill Meta Data

In [5]:
from pathlib import Path
from skills_ref import parser
from skills_ref.errors import ParseError, ValidationError

print("=" * 80)
print("TESTING ALL PARSER FUNCTIONS")
print("=" * 80)

# Test 1: find_skill_md()
print("\n[1] Testing find_skill_md()")
print("-" * 80)
skill_dirs = [
    Path("/home/snt/projects_lujun/LabAgentSkill/skillsHub/skills/langgraph-docs"),
    Path("/home/snt/projects_lujun/LabAgentSkill/skillsHub/skills/sales-analytics"),
]

for skill_dir in skill_dirs:
    result = parser.find_skill_md(skill_dir)
    print(f"Directory: {skill_dir.name}")
    print(f"  Result: {result}")
    if result:
        print(f"  File exists: {result.exists()}")

# Test non-existent directory
fake_dir = Path("/home/snt/projects_lujun/LabAgentSkill/skillsHub/skills/nonexistent")
result = parser.find_skill_md(fake_dir)
print(f"Directory: nonexistent (fake)")
print(f"  Result: {result}")

# Test 2: parse_frontmatter()
print("\n[2] Testing parse_frontmatter()")
print("-" * 80)

# Valid YAML frontmatter
valid_content = """---
name: test-skill
description: A test skill for parsing
license: MIT
compatibility: v1.0
allowed-tools:
  - tool1
  - tool2
metadata:
  version: "1.0"
  author: "test"
---

# Skill Body

This is the markdown body of the skill."""

try:
    metadata, body = parser.parse_frontmatter(valid_content)
    print("✓ Valid frontmatter parsed successfully")
    print(f"  Metadata keys: {list(metadata.keys())}")
    print(f"  Body preview: {body[:50]}...")
except Exception as e:
    print(f"✗ Error: {e}")

# Invalid: missing closing ---
invalid_content = """---
name: test-skill
description: test

# No closing ---
"""

try:
    metadata, body = parser.parse_frontmatter(invalid_content)
    print("✗ Should have raised ParseError")
except ParseError as e:
    print(f"✓ Correctly raised ParseError: {e}")

# Invalid: doesn't start with ---
no_frontmatter = "# Just markdown"
try:
    metadata, body = parser.parse_frontmatter(no_frontmatter)
    print("✗ Should have raised ParseError")
except ParseError as e:
    print(f"✓ Correctly raised ParseError: {e}")

# Test 3: read_properties()
print("\n[3] Testing read_properties()")
print("-" * 80)

for skill_dir in skill_dirs:
    try:
        props = parser.read_properties(skill_dir)
        print(f"✓ {skill_dir.name}")
        print(f"  Name: {props['name']}")
        print(f"  Description: {props['description'][:60]}...")
        print(f"  License: {props.get('license', 'N/A')}")
        print(f"  Allowed Tools: {props.get('allowed-tools', 'N/A')}")
    except Exception as e:
        print(f"✗ {skill_dir.name}: {type(e).__name__}: {e}")

# Test with non-existent directory
print("\nTesting with non-existent directory:")
try:
    props = parser.read_properties(fake_dir)
    print("✗ Should have raised ParseError")
except ParseError as e:
    print(f"✓ Correctly raised ParseError: {e}")

# Test 4: Frontmatter parsing with various valid YAML
print("\n[4] Testing frontmatter with complex YAML")
print("-" * 80)

complex_content = """---
name: complex-skill
description: Complex skill with nested structures
allowed-tools:
  - sql_query
  - api_call
metadata:
  version: "2.0"
  tags:
    - database
    - analytics
  authors:
    - Alice
    - Bob
---

# Complex Skill Body
Content here."""

try:
    metadata, body = parser.parse_frontmatter(complex_content)
    print("✓ Complex YAML parsed successfully")
    print(f"  Name: {metadata['name']}")
    print(f"  Tools: {metadata['allowed-tools']}")
    print(f"  Metadata: {metadata['metadata']}")
except Exception as e:
    print(f"✗ Error: {e}")

# Test 5: Edge cases
print("\n[5] Testing edge cases")
print("-" * 80)

# Minimal valid frontmatter
minimal = """---
name: minimal
description: minimal skill
---

Body"""

try:
    metadata, body = parser.parse_frontmatter(minimal)
    print("✓ Minimal frontmatter parsed")
    print(f"  Keys: {list(metadata.keys())}")
except Exception as e:
    print(f"✗ Error: {e}")

# Empty body
empty_body = """---
name: test
description: test
---
"""

try:
    metadata, body = parser.parse_frontmatter(empty_body)
    print("✓ Empty body handled correctly")
    print(f"  Body is empty: {body == ''}")
except Exception as e:
    print(f"✗ Error: {e}")

print("\n" + "=" * 80)
print("PARSER TESTS COMPLETE")
print("=" * 80)

TESTING ALL PARSER FUNCTIONS

[1] Testing find_skill_md()
--------------------------------------------------------------------------------
Directory: langgraph-docs
  Result: /home/snt/projects_lujun/LabAgentSkill/skillsHub/skills/langgraph-docs/SKILL.md
  File exists: True
Directory: sales-analytics
  Result: /home/snt/projects_lujun/LabAgentSkill/skillsHub/skills/sales-analytics/SKILL.md
  File exists: True
Directory: nonexistent (fake)
  Result: None

[2] Testing parse_frontmatter()
--------------------------------------------------------------------------------
✓ Valid frontmatter parsed successfully
  Metadata keys: ['name', 'description', 'license', 'compatibility', 'allowed-tools', 'metadata']
  Body preview: # Skill Body

This is the markdown body of the ski...
✗ Should have raised ParseError
✓ Correctly raised ParseError: SKILL.md must start with YAML frontmatter (---)

[3] Testing read_properties()
------------------------------------------------------------------------------

In [7]:
from pathlib import Path
from skills_ref import validator, prompt

print("\n" + "=" * 80)
print("TESTING VALIDATOR AND PROMPT MODULES")
print("=" * 80)

# Test 1: validator._validate_name()
print("\n[1] Testing validator._validate_name()")
print("-" * 80)

test_cases_name = [
    ("valid-skill", Path("/test/valid-skill"), True),
    ("ValidSkill", Path("/test/ValidSkill"), False),  # Not lowercase
    ("skill-", Path("/test/skill-"), False),  # Ends with hyphen
    ("-skill", Path("/test/-skill"), False),  # Starts with hyphen
    ("skill--name", Path("/test/skill--name"), False),  # Double hyphen
    ("skill@name", Path("/test/skill@name"), False),  # Invalid character
    ("x" * 65, Path("/test/x" * 65), False),  # Too long
    ("valid-skill", Path("/test/wrong-dir"), False),  # Dir name mismatch
]

for name, skill_dir, should_pass in test_cases_name:
    errors = validator._validate_name(name, skill_dir)
    status = "✓ PASS" if (not errors) == should_pass else "✗ FAIL"
    print(f"{status}: {name[:40]}")
    if errors:
        for error in errors[:2]:  # Show first 2 errors
            print(f"    - {error}")

# Test 2: validator._validate_description()
print("\n[2] Testing validator._validate_description()")
print("-" * 80)

test_cases_desc = [
    ("A valid description", True),
    ("", False),  # Empty
    ("x" * 1025, False),  # Too long
    (123, False),  # Not a string
    ("Short but valid", True),
]

for desc, should_pass in test_cases_desc:
    errors = validator._validate_description(desc)
    status = "✓ PASS" if (not errors) == should_pass else "✗ FAIL"
    desc_preview = str(desc)[:40] if isinstance(desc, str) else str(type(desc).__name__)
    print(f"{status}: {desc_preview}")
    if errors:
        print(f"    - {errors[0]}")

# Test 3: validator._validate_compatibility()
print("\n[3] Testing validator._validate_compatibility()")
print("-" * 80)

test_cases_compat = [
    ("v1.0", True),
    ("v1.0 - v2.0", True),
    ("x" * 501, False),  # Too long
    (123, False),  # Not a string
]

for compat, should_pass in test_cases_compat:
    errors = validator._validate_compatibility(compat)
    status = "✓ PASS" if (not errors) == should_pass else "✗ FAIL"
    compat_preview = str(compat)[:40] if isinstance(compat, str) else str(type(compat).__name__)
    print(f"{status}: {compat_preview}")
    if errors:
        print(f"    - {errors[0]}")

# Test 4: prompt.to_prompt()
print("\n[4] Testing prompt.to_prompt()")
print("-" * 80)

# Test with empty list
skill_dirs_empty = []
xml_output = prompt.to_prompt(skill_dirs_empty)
print("Empty skill list:")
print(f"  Output: {xml_output}")

# Test with actual skill directories
skill_dirs = [
    Path("/home/snt/projects_lujun/LabAgentSkill/skillsHub/skills/langgraph-docs"),
    Path("/home/snt/projects_lujun/LabAgentSkill/skillsHub/skills/sales-analytics"),
]

print("\nWith existing skills:")
xml_output = prompt.to_prompt(skill_dirs)
lines = xml_output.split('\n')
print(f"  Total lines: {len(lines)}")
print(f"  Contains <available_skills>: {'<available_skills>' in xml_output}")
print(f"  Contains <skill>: {xml_output.count('<skill>')}")
print(f"  XML structure:")
for line in lines[:15]:  # Show first 15 lines
    print(f"    {line}")

# Verify XML is well-formed (basic check)
skill_count = xml_output.count("<skill>")
close_count = xml_output.count("</skill>")
print(f"  Matching tags: {skill_count} == {close_count}: {skill_count == close_count}")

# Test 5: Test with HTML escaping
print("\n[5] Testing HTML escaping in prompt.to_prompt()")
print("-" * 80)

# Create a mock skill with special characters
class MockSkillProperties:
    def __init__(self, name, description):
        self.name = name
        self.description = description
    
    def __getitem__(self, key):
        if key == 'name':
            return self.name
        elif key == 'description':
            return self.description

# Test that special characters are escaped
test_desc_with_special = "Description with <tags> & 'quotes' \"double\""
print(f"Original: {test_desc_with_special}")

# The actual escaping happens in prompt.to_prompt via html.escape
import html
escaped = html.escape(test_desc_with_special)
print(f"Escaped:  {escaped}")
print(f"  Correctly escaped: {'&lt;' in escaped and '&amp;' in escaped}")

print("\n" + "=" * 80)
print("VALIDATOR & PROMPT TESTS COMPLETE")
print("=" * 80)


TESTING VALIDATOR AND PROMPT MODULES

[1] Testing validator._validate_name()
--------------------------------------------------------------------------------
✓ PASS: valid-skill
✓ PASS: ValidSkill
    - Skill name 'ValidSkill' must be lowercase
✓ PASS: skill-
    - Skill name cannot start or end with a hyphen
✓ PASS: -skill
    - Skill name cannot start or end with a hyphen
✓ PASS: skill--name
    - Skill name cannot contain consecutive hyphens
✓ PASS: skill@name
    - Skill name 'skill@name' contains invalid characters. Only letters, digits, and hyphens are allowed.
✓ PASS: xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
    - Skill name 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' exceeds 64 character limit (65 chars)
    - Directory name 'x' must match skill name 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
✓ PASS: valid-skill
    - Directory name 'wrong-dir' must match skill name 'valid-skill'

[2] Testing validator._validate_description()
--